<a href="https://colab.research.google.com/github/Kiddy204/Assignment_Angular/blob/master/Assignements/Part%204/Assignment_part_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### DSML investigation

You are part of the Suisse Impossible Mission Force, or SIMF for short. You need to uncover a rogue agent that is trying to steal sensitive information.

Your mission, should you choose to accept it, is to find that agent before stealing any classified information. Good luck!

# Assignement part four

#### Identifying the suspects' credit score
We received informations that the rogue agent has a *good* credit score.

Our spies at SIMF have managed to collect financial information relating to our suspects as well as a training dataset.

Create a Neural Network over the training dataset `df` to identify which of the suspects have a *standard* `Credit_Mix`.


## Getting to know our data

* `Age`: a user's age

* `Occupation`: a user's employment field

* `Annual_Income`: a user's annual income

* `Monthly_Inh_Salary`: the calculated salary received by a given user on a monthly basis

* `Num_Bank_Accounts`: the number of bank accounts possessed by a given user

* `Num_Credit_Cards`: the number of credit cards a given user possesses

* `Interest_Rate`: The interest rate on those cards (if multiple then it's the average)

* `Num_of_Loans`: The number of loans of each user

* `Delay_from_due_date`: payment tardiness of user

* `Num_of_Delayed_Payment`: the count of delayed payments

* `Changed_Credit_Limit`: NaN

* `Num_Credit_Inquiries`: NaN

* `Credit_Mix`: The user's credit score

* `Outsting_Debt`: Outstanding debt

* `Credit_Utilization_Ratio`: the percentage of borrowed money over borrowing allowance

* `Payment_of_Min_Amount`: does the user usually pay the minimal amount (categorical)

* `Total_EMI_per_month`: Monthly repayments to be made

* `Amount_invested_monthly`: The amount put in an investment fund by the user on a monthly basis

* `Payment_Behaviour`: the user's payment behavior (categorical)

* `Monthly_Balance`: The user's end of the month balance

* `AutoLoan`: If the user has an active loan for their vehicle

* `Credit-BuilderLoan`: If the user has a loan to increase their credit score

* `DebtConsolidationLoan`, `HomeEquityLoan`, `MortgageLoan`, `NotSpecified`, `PaydayLoan`, `PersonalLoan`, `StudentLoan`: different types of loans (categorical features)


In [1]:
# Import required packages

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [15]:
df = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/train_classification.csv", index_col='Unnamed: 0').dropna()
suspects = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/suspects.csv", index_col='Unnamed: 0').dropna()
suspects.rename(columns={"Payment_Behaviour": "le_Payment_Behaviour", "Payment_of_Min_Amount": "le_Payment_of_Min_Amount"}, inplace=True)

In [16]:
display(df.head())
print(df.shape)
display(suspects.head())
print(suspects.shape)

,Age,Occupation,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Monthly_Balance,AutoLoan,Credit-BuilderLoan,DebtConsolidationLoan,HomeEquityLoan,MortgageLoan,NotSpecified,PaydayLoan,PersonalLoan,StudentLoan
0,23,Scientist,19114.12,1824.843333,3,4,3,4,3,7,...,186.266702,1,1,0,1,0,0,0,1,0
1,24,Scientist,19114.12,1824.843333,3,4,3,4,3,9,...,361.444004,1,1,0,1,0,0,0,1,0
3,24,Scientist,19114.12,4182.004291,3,4,3,4,4,5,...,343.826873,1,1,0,1,0,0,0,1,0
5,28,Teacher,34847.84,3037.986667,2,4,6,1,3,3,...,303.355083,0,1,0,0,0,0,0,0,0
8,35,Engineer,143162.64,4182.004291,1,5,8,3,8,1942,...,854.226027,2,0,0,0,0,1,0,0,0


(29223, 29)


,Age,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_MediaManager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,userID
0,23.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,7.0,11.27,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,317991
1,24.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,9.0,13.27,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,241892
3,24.0,19114.12,4182.004291,3.0,4.0,3.0,4.0,4.0,5.0,11.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,303376
5,28.0,34847.84,3037.986667,2.0,4.0,6.0,1.0,3.0,3.0,5.42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,761992
8,35.0,143162.64,4182.004291,1.0,5.0,8.0,3.0,8.0,1942.0,7.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,373318


(714, 43)


In [17]:
df["Credit_Mix"].unique()

array(['Good', 'Standard', 'Bad'], dtype=object)

# 1. Preparing the data
## 1.1 Data cleaning
Consider the dataset loaded into the DataFrame `df` which is *train_classification.csv*. We aim to preprocess this data for model training.

Begin by encoding the categorical variables:
- Apply One-Hot Encoding to `Occupation`
- Apply Label Encoding to `Payment_of_Min_Amount` and `Payment_Behaviour`

*Note: To clearly distinguish your newly encoded columns, especially for label encoding, consider renaming them with a prefix. Please, use `le_Payment_of_Min_Amount` and `le_Payment_Behaviour` for the label-encoded new columns.*

In [18]:
# Your code here
ohe = OneHotEncoder(sparse_output=False)
ohe_fit_occupation = ohe.fit(df[['Occupation']])
occupation_encoded = ohe_fit_occupation.transform(df[['Occupation']])
occupation_encoded = pd.DataFrame(
    occupation_encoded,
    columns=ohe_fit_occupation.get_feature_names_out(),
    index=df.index

  )

le_payment_min = LabelEncoder()


le_payment_behaviour = LabelEncoder()



 After encoding, integrate the newly encoded columns back into a new DataFrame named `df_encoded`, and remove the original columns `Occupation`, `Payment_of_Min_Amount`, and `Payment_Behaviour` to avoid redundancy.

In [19]:
# Your code here
df_encoded = df.copy()

In [37]:
print(df.shape)
print(df_encoded.shape)
print(suspects.shape)
suspects.head()

(29223, 29)
(29223, 43)
(714, 43)


,Age,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_MediaManager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,userID
0,23.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,7.0,11.27,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,317991
1,24.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,9.0,13.27,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,241892
3,24.0,19114.12,4182.004291,3.0,4.0,3.0,4.0,4.0,5.0,11.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,303376
5,28.0,34847.84,3037.986667,2.0,4.0,6.0,1.0,3.0,3.0,5.42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,761992
8,35.0,143162.64,4182.004291,1.0,5.0,8.0,3.0,8.0,1942.0,7.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,373318


In [21]:
df_encoded['le_Payment_of_Min_Amount'] = le_payment_min.fit_transform(df['Payment_of_Min_Amount'])
df_encoded = df_encoded.drop('Payment_of_Min_Amount', axis=1)

df_encoded['le_Payment_Behaviour'] = le_payment_behaviour.fit_transform(df['Payment_Behaviour'])
df_encoded = df_encoded.drop('Payment_Behaviour', axis=1)

In [23]:
df_encoded = pd.concat([df_encoded.drop('Occupation', axis=1), occupation_encoded], axis=1)


Finally, display the first few rows of `df_encoded` to verify that the encodings are correctly implemented. This prepared DataFrame will be used for subsequent model training.

In [25]:
# Your code here
df_encoded.head()

,Age,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_MediaManager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer
0,23,19114.12,1824.843333,3,4,3,4,3,7,11.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,24,19114.12,1824.843333,3,4,3,4,3,9,13.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,24,19114.12,4182.004291,3,4,3,4,4,5,11.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,28,34847.84,3037.986667,2,4,6,1,3,3,5.42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,35,143162.64,4182.004291,1,5,8,3,8,1942,7.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1.2 Dataset splitting and rescaling

To effectively train and validate our model, it is crucial to properly prepare and partition the data. Follow these steps to preprocess and split the dataframe df_encoded into training and test subsets, ensuring that our model can generalize well to new data:

- Set `X` as all columns except `Credit_Mix` and `y` as the dependent feature `Credit_Mix`.
- Apply manually Label Encoding to `y` using the function `.map`and encoding with the following : Good=2, Standard=1, Bad=0.
- Use a `random_state` of 42 to split `X` and the encoded `y` into training (80%) and test sets (20%).
- Normalize `X` using `MinMaxScaler()`.

In [26]:
# Your code here
X = df_encoded.drop('Credit_Mix', axis=1)
y = df_encoded['Credit_Mix']

y = y.map({ 'Good': 2, 'Standard': 1, 'Bad': 0 })

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### 1.2.2 Final touches
Convert the features to torch tensors of type `torch.float` and the labels (dependent variables) to torch tensors of type `torch.long`.

In [28]:
#Your code here
X_train = torch.tensor(X_train, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)

# Convert labels to torch tensors of type torch.long
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# 2 Model preparation:

## 2.1 Define a Neural network model and instantiate it.
Define your neural network model as a class in PyTorch, extending from nn.Module. In the `__init__` method, initialize a linear layer using `nn.Linear()` with specified input and output sizes, and set up `nn.ReLU()` for activation. Implement the forward method to describe how data passes through this layer during the network's forward computation.

Set the following parameters:
* `hidden layer` : 1
* `activation function` : ReLU

In [29]:
# Your code here


class NET(nn.Module):
  def __init__(self, input_dim:int, hidden_dim: int, output_dim: int):
    super(NET,self).__init__()
    self.linear1 = nn.Linear(input_dim,hidden_dim)
    self.activation = nn.ReLU()
    self.linear2 = nn.Linear(hidden_dim, output_dim)

  def forward(self, x: torch.tensor):
    hidden= self.activation(self.linear1(x))
    y_pred= self.linear2(hidden)
    return y_pred

Set `D_in` to the number of features in `X_train` and `D_out` to the number of target variables in `y_train`, then print these dimensions to verify their values.

In [30]:
# Your code here
D_in = X_train.shape[1]   # number of input features
D_out = len(y_train.unique())  # number of target classes


model = NET(input_dim=D_in, hidden_dim=1, output_dim=D_out)
print(f"D_in = {D_in}")
print(f"D_out = {D_out}")
print(model)

D_in = 42
D_out = 3
NET(
  (linear1): Linear(in_features=42, out_features=1, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=1, out_features=3, bias=True)
)



Initialize the `Net` model with the specified input size `D_in`, 150 hidden units, and output size `D_out`.

In [31]:
# Your code here
model = NET(input_dim=D_in, hidden_dim=150, output_dim=D_out)
print(model)

NET(
  (linear1): Linear(in_features=42, out_features=150, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=150, out_features=3, bias=True)
)


Let's calculate now how many parameters we have in the model.

In [32]:
# Your code here
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

6903


**Q1. How many parameters does your model have ?**

*Note: Enter an integer (e.g. 355)*

6903

## 2.2 Finding the best model

Determine the optimal hyper-parameters for your model from the options listed below:

* `criterion` : [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
* `optimizer` : [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)
* `Epochs`: Test with **150**, **250**, **500**, and **1000** epochs.
* `Learning Rate`: Experiment with learning rates of **0.00005**, **0.001**, **1**, and **10**.

**Evaluation**: Assess your model's performance by measuring its accuracy on the test set.

*Note: Run the code `torch.manual_seed(42)` to ensure consistency across all experiments.*

In [33]:
torch.manual_seed(42)   # Set the seed for reproducibility

### 2.2.1 Automatically Tuning Hyperparameters

In this section, you will automate the process of testing different hyperparameter combinations using a loop, as demonstrated in the lab.

Begin by defining the ranges for the hyperparameters you want to explore:
- **Epochs**: Test with **150**, **250**, **500**, and **1000** epochs.
- **Learning Rate**: Experiment with learning rates of **0.00005**, **0.001**, **1**, and **10**.


In [34]:
# Your code here
epoch_list = [150, 250, 500, 1000]
lr_list = [0.00005, 0.001, 1, 10]

Next, create a loop that iterates over the list of learning rates, with an inner loop iterating over the list of number of epochs. Inside the inner loop, initialize (define) the model, and also define the optimizer and the loss function (criterion). Then train the model as demonstrated in the lab, and after the training evaluate its performance to obtain the test accuracy for each model. Ensure that you also display the test loss, as you will need it to answer the upcoming questions.

Here’s a reminder of the criterion and optimizer you should use:
- **Criterion**: [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
- **Optimizer**: [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)

**Hint**: Make sure to define the criterion and optimizer at every iteration (inside the inner loop), otherwise the model might retain previous training states and produce biased results.

(Optional) You can enhance the code by storing the best model along with its best_accuracy and best_params. This way, you'll have an automatic evaluation at the end to identify the best-performing model.



In [35]:
# Your code here
criterion = nn.CrossEntropyLoss()

results = {}

# -------------------------
# Loop over combinations
# -------------------------
for epochs in epoch_list:
    for lr in lr_list:
        # Initialize model
        model = NET(input_dim=D_in, hidden_dim=150, output_dim=D_out)

        # Optimizer
        optimizer = optim.SGD(model.parameters(), lr=lr)

        losses_train = []
        losses_test = []

        for t in range(epochs):
            # Forward pass
            y_pred = model(X_train)
            loss = criterion(y_pred, y_train)
            losses_train.append(loss.item())

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Evaluate on test set
            with torch.no_grad():
                test_loss = criterion(model(X_test), y_test).item()
                losses_test.append(test_loss)

        # Compute final accuracy on test set
        with torch.no_grad():
            y_pred_test = model(X_test)
            y_pred_labels = torch.argmax(y_pred_test, dim=1)
            accuracy = (y_pred_labels == y_test).float().mean().item()

        # Store results
        results[(epochs, lr)] = accuracy
        print(f"Epochs={epochs}, LR={lr} --> Accuracy={accuracy:.4f}")

# -------------------------
# Display best combination
# -------------------------
best_params = max(results, key=results.get)
print(f"\nBest Hyperparameters:\nEpochs={best_params[0]}, LR={best_params[1]} with Accuracy={results[best_params]:.4f}")

Epochs=150, LR=5e-05 --> Accuracy=0.3092
Epochs=150, LR=0.001 --> Accuracy=0.3293
Epochs=150, LR=1 --> Accuracy=0.8118
Epochs=150, LR=10 --> Accuracy=0.4546
Epochs=250, LR=5e-05 --> Accuracy=0.3514
Epochs=250, LR=0.001 --> Accuracy=0.4559
Epochs=250, LR=1 --> Accuracy=0.8323
Epochs=250, LR=10 --> Accuracy=0.4546
Epochs=500, LR=5e-05 --> Accuracy=0.2315
Epochs=500, LR=0.001 --> Accuracy=0.4546
Epochs=500, LR=1 --> Accuracy=0.8378
Epochs=500, LR=10 --> Accuracy=0.4546
Epochs=1000, LR=5e-05 --> Accuracy=0.4212
Epochs=1000, LR=0.001 --> Accuracy=0.4546
Epochs=1000, LR=1 --> Accuracy=0.8465
Epochs=1000, LR=10 --> Accuracy=0.3145

Best Hyperparameters:
Epochs=1000, LR=1 with Accuracy=0.8465


### 2.2.3 Questions

**Q2. What is the test accuracy when we train the model with a learning rate of 0.001 and for 150 epochs? Round your answer to 2 decimal points, e.g., 0.25.**

0.33

**Q3. When using 1000 epochs, which learning rate results in the highest test accuracy?**

*Note: Select among the following answers*

LR = 1

[link text](https://)
**Q4. Is BCELoss a suitable alternative to CrossEntropyLoss for our dataset?**

*Hint: Consider the unique values in the Credit_Mix output variable when answering.*

No: No because there are three classes. Not suitable for Binary class enthropy

### 3. Predict on the Suspects Dataset

Now it's time to use your trained model to make predictions on the suspects dataset!

Please retrain on the full dataset.

Use the following parameters for the model:
- **Hidden layer**: 1 hidden layer with 150 neurons
- **Output layer**: 3 neurons for classification
- **Optimizer**: [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)
- **Criterion**: [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
- **Iterations**: 1000
- **Learning rate**: 1.0

Ensure consistency by setting the manual seed with `torch.manual_seed(42)` before training.

In [57]:


# Your code here
# DROP 'UserID columns'
user_ids = suspects["userID"].copy()
suspects_input = suspects.drop(columns=["userID"])

df_encoded['Credit_Mix'].head()
X_full = df_encoded.drop('Credit_Mix', axis=1)
y_full = df_encoded['Credit_Mix']
y_full = y_full.map({ 'Good': 2, 'Standard': 1, 'Bad': 0 })

# Align Suspsect columns with X columns:
suspects_input = suspects_input.reindex(columns=X_full.columns, fill_value=0)
# Scale features for train and test dataframes
scaler = MinMaxScaler()
X_full_scaled = scaler.fit_transform(X_full)
suspects_scaled = scaler.transform(suspects_input)

X_full_t = torch.tensor(X_full_scaled, dtype=torch.float32)
y_full_t = torch.tensor(y_full.values, dtype=torch.long)
suspects_t = torch.tensor(suspects_scaled, dtype=torch.float32)

D_in = X_full_t.shape[1]
D_out = len(y_full_t.unique())

In [62]:
#print(suspects.shape)
#print(suspects.columns)

#print(X_full.shape)
#print(X_full.columns)
torch.manual_seed(42)

model_full = NET(input_dim=D_in, hidden_dim=150, output_dim=D_out)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_full.parameters(), lr=1.0)



Now, train your model on the training dataset just as you did in section 2.2.1, but you don't have to loop over all the hyper parameters. Ensure that you use the correct number of epochs specified earlier.

In [63]:
# Your code here

optimizer = optim.SGD(model_full.parameters(), lr=1.0)

n_epochs = 1000

for epoch in range(n_epochs):
    optimizer.zero_grad()
    outputs = model_full(X_full_t)
    loss = criterion(outputs, y_full_t)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        _, predicted = torch.max(outputs, 1)
        acc = (predicted == y_full_t).float().mean().item()
        print(f"Epoch [{epoch+1}/{n_epochs}] - Loss: {loss.item():.4f} - Accuracy: {acc:.4f}")



Epoch [100/1000] - Loss: 0.4449 - Accuracy: 0.8013
Epoch [200/1000] - Loss: 0.3736 - Accuracy: 0.8324
Epoch [300/1000] - Loss: 0.3603 - Accuracy: 0.8357
Epoch [400/1000] - Loss: 0.3487 - Accuracy: 0.8404
Epoch [500/1000] - Loss: 0.3446 - Accuracy: 0.8420
Epoch [600/1000] - Loss: 0.3384 - Accuracy: 0.8466
Epoch [700/1000] - Loss: 0.3359 - Accuracy: 0.8464
Epoch [800/1000] - Loss: 0.3316 - Accuracy: 0.8484
Epoch [900/1000] - Loss: 0.3275 - Accuracy: 0.8510
Epoch [1000/1000] - Loss: 0.3303 - Accuracy: 0.8474


Before making predictions, confirm that the feature column names in the `suspects` dataset match those expected by the model, particularly ensuring the X features correspond accurately.

In [55]:
print("Unique values in y_full_t:", torch.unique(y_full_t))
print("Data type of y_full_t:", y_full_t.dtype)
print("Label distribution:", torch.bincount(y_full_t))


Unique values in y_full_t: tensor([0, 1, 2])
Data type of y_full_t: torch.int64
Label distribution: tensor([ 6839, 13421,  8963])


Then scale your dataset and convert it into a torch tensor of dtype float, similar to the preprocessing done for the training set in section 1.2.

In [ ]:
#Your code here

Make predictions using the trained model and assign the predicted credit score to each user. Ensure to do the following encoding is used for the predicted categories:
* `0` corresponds to bad credit score,
* `1` corresponds to standard credit score,
* `2` corresponds to good credit score.

Use the predictions to add a new column `credit_score` in the `suspects` dataframe that maps the predicted numerical values to the respective credit score categories.

In [64]:
# Ensure the model is in evaluation mode
model_full.eval()

# Make predictions on the suspects dataset
with torch.no_grad():
    outputs = model_full(suspects_t)
    _, predicted_classes = torch.max(outputs, 1)

# Convert predictions to numpy
predicted_classes_np = predicted_classes.numpy()

# Map numeric predictions to credit score categories
credit_score_mapping = {0: "Bad", 1: "Standard", 2: "Good"}
suspects["credit_score"] = [credit_score_mapping[p] for p in predicted_classes_np]

# Add back user IDs
suspects["userID"] = user_ids.values

# Display results
suspects[["userID", "credit_score"]].head()


,userID,credit_score
0,317991,Good
1,241892,Good
3,303376,Good
5,761992,Good
8,373318,Good




As mentioned earlier, we believe the suspect had a "good" credit score. Review the predictions made by the model, then display how many suspects were categorized under each credit score, and extract the `userID`s of those with a "standard" credit score.

In [65]:
# Your code here

credit_score_counts = suspects["credit_score"].value_counts()
print("Predicted Credit Score Distribution:\n", credit_score_counts)

# Extract userIDs of suspects with "Standard" credit score
standard_users = suspects.loc[suspects["credit_score"] == "Standard", "userID"].tolist()

print(f"\nNumber of suspects with 'Standard' credit score: {len(standard_users)}")
print("\nUserIDs of suspects with 'Standard' credit score:")
print(standard_users)

Predicted Credit Score Distribution:
 credit_score
Standard    272
Good        268
Bad         174
Name: count, dtype: int64

Number of suspects with 'Standard' credit score: 272

UserIDs of suspects with 'Standard' credit score:
[660506, 244162, 353527, 531937, 413250, 512604, 334769, 564061, 306495, 647912, 946059, 917315, 504629, 520229, 861915, 308845, 888670, 449514, 867078, 618568, 512128, 777470, 397912, 123562, 939524, 863210, 717780, 863592, 898254, 424603, 599462, 601265, 402663, 551506, 365406, 688784, 219047, 380835, 823355, 456546, 790046, 932281, 571364, 796491, 205775, 133154, 430864, 899581, 240613, 126115, 839591, 453712, 259083, 512432, 874150, 484182, 291359, 744124, 641947, 781819, 666304, 619595, 245167, 893389, 953342, 570734, 743377, 510420, 204180, 564884, 107841, 937535, 376743, 767527, 225178, 519735, 706286, 934741, 792996, 140991, 641389, 241404, 325946, 154013, 537543, 875439, 418245, 843485, 99489, 231948, 645264, 649131, 509130, 719655, 316652, 726678, 80

**Q5.Which of the following suspects have a "good" credit mix according to your model's predictions?**


## Your investigation is progressing effectively, and the list of suspects is narrowing down.

**Don't forget to answer the quiz and submit your code on Moodle before the end of the deadline.**